In [14]:
import pandas as pd

data_files = {
    "Categories": "../basededatos/categories.csv",
    "Customers": "../basededatos/customers.csv",
    "Employees": "../basededatos/employees.csv",
    "Orders": "../basededatos/orders.csv",
    "Order details": "../basededatos/order_details.csv",
    "Products": "../basededatos/products.csv",
    "Suppliers": "../basededatos/suppliers.csv"
}

for name, file_path in data_files.items():
    data = pd.read_csv(file_path, header=0, index_col=False)
    print(f"{name}:")
    display(data.head())

Categories:


,CategoryID,CategoryName,Description,Picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales","{""type"":""Buffer"",""data"":[]}"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an...","{""type"":""Buffer"",""data"":[]}"
2,3,Confections,"Desserts, candies, and sweet breads","{""type"":""Buffer"",""data"":[]}"
3,4,Dairy Products,Cheeses,"{""type"":""Buffer"",""data"":[]}"
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal","{""type"":""Buffer"",""data"":[]}"


Customers:


,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NaN,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,NaN,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,NaN,05023,Mexico,(5) 555-3932,NaN
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,NaN,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,NaN,S-958 22,Sweden,0921-12 34 65,0921-12 34 67


Employees:


...

,OrderID,ProductID,UnitPrice,Quantity,Discount
0,10248,11,14.0,12,0.0
1,10248,42,9.8,10,0.0
2,10248,72,34.8,5,0.0
3,10249,14,18.6,9,0.0
4,10249,51,42.4,40,0.0


Products:


,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,8,1,10 boxes x 30 bags,18.00,39,0,10,1
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,1
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


Suppliers:


,SupplierID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,NaN,EC1 4SD,UK,(171) 555-2222,NaN,NaN
1,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,LA,70117,USA,(100) 555-4822,NaN,#CAJUN.HTM#
2,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,MI,48104,USA,(313) 555-5735,(313) 555-3349,NaN
3,4,Tokyo Traders,Yoshi Nagase,Marketing Manager,9-8 Sekimai Musashino-shi,Tokyo,NaN,100,Japan,(03) 3555-5011,NaN,NaN
4,5,Cooperativa de Quesos 'Las Cabras',Antonio del Valle Saavedra,Export Administrator,Calle del Rosal 4,Oviedo,Asturias,33007,Spain,(98) 598 76 54,NaN,NaN


ventas.csv

orderid,fecha,empleado,producto,idproducto,cantidad,preccioventa,descuento,importe,categoria,proveedor,ciudad_cliente,pais_cliente,cliente

In [15]:
def merge_dataframes(orders, employees, order_details, products, categories, suppliers, customers):
    return (
        orders.merge(employees, on='EmployeeID', how='left')
        .merge(order_details, on='OrderID', how='inner')
        .merge(products, on='ProductID', how='left')
        .merge(categories, on='CategoryID', how='left')
        .merge(suppliers, on='SupplierID', how='left')
        .merge(customers, on='CustomerID', how='left')
    )

def calculate_importe(result):
    return ((result['UnitPrice_x'] - (result['Discount'] * result['UnitPrice_x'])) * result['Quantity']).round(2)

merged_result = merge_dataframes(orders, employees, order_details, products, categories, suppliers, customers)

merged_result['importe'] = calculate_importe(merged_result)

final_result = merged_result[[
    'OrderID',
    'OrderDate',
    'FirstName',
    'LastName',
    'ProductName',
    'ProductID',
    'Quantity',
    'UnitPrice_x',
    'Discount',
    'importe',
    'CategoryName',
    'CompanyName_x',
    'City',
    'Country',
    'ContactName_y'
]].rename(columns={
    'OrderID': 'orderid',
    'OrderDate': 'fecha',
    'FirstName': 'empleado',
    'LastName': 'Apellido',  # Puedes combinar FirstName y LastName si lo prefieres
    'ProductName': 'producto',
    'ProductID': 'idproducto',
    'Quantity': 'cantidad',
    'UnitPrice_x': 'preccioventa',
    'Discount': 'descuento',
    'importe': 'importe',
    'CategoryName': 'categoria',
    'CompanyName_x': 'proveedor',
    'City': 'ciudad_cliente',
    'Country': 'pais_cliente',
    'ContactName_y': 'cliente'
})


display(final_result)


,orderid,fecha,Nombre,Apellido,producto,idproducto,cantidad,preccioventa,descuento,importe,categoria,proveedor,ciudad_cliente,pais_cliente,cliente
0,10248,1996-07-04,Steven,Buchanan,Queso Cabrales,11,12,14.0,0.00,168.00,Dairy Products,Cooperativa de Quesos 'Las Cabras',Reims,France,Paul Henriot
1,10248,1996-07-04,Steven,Buchanan,Singaporean Hokkien Fried Mee,42,10,9.8,0.00,98.00,Grains/Cereals,Leka Trading,Reims,France,Paul Henriot
2,10248,1996-07-04,Steven,Buchanan,Mozzarella di Giovanni,72,5,34.8,0.00,174.00,Dairy Products,Formaggi Fortini s.r.l.,Reims,France,Paul Henriot
3,10249,1996-07-05,Michael,Suyama,Tofu,14,9,18.6,0.00,167.40,Produce,Mayumi's,Münster,Germany,Karin Josephs
4,10249,1996-07-05,Michael,Suyama,Manjimup Dried Apples,51,40,42.4,0.00,1696.00,Produce,"G'day, Mate",Münster,Germany,Karin Josephs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10284,1996-08-19,Margaret,Peacock,Schoggi Schokolade,27,15,35.1,0.25,394.88,Confections,Heli Süßwaren GmbH & Co. KG,Frankfurt a.M.,Germany,Renate Messner
96,10284,1996-08-19,Margaret,Peacock,Gula Malacca,44,21,15.5,0.00,325.50,Condiments,Leka Trading,Frankfurt a.M.,Germany,Renate Messner
97,10284,1996-08-19,Margaret,Peacock,Camembert Pierrot,60,20,27.2,0.25,408.00,Dairy Products,Gai pâturage,Frankfurt a.M.,Germany,Renate Messner
98,10284,1996-08-19,Margaret,Peacock,Laughing Lumberjack Lager,67,5,11.2,0.25,42.00,Beverages,Bigfoot Breweries,Frankfurt a.M.,Germany,Renate Messner


ventas_tiempos.csv

cliente,fecha_venta,fecha_venta_anterior,diferencia_dias,diferencia_meses,monto_venta

In [21]:
import pandas as pd

# Función para convertir fechas
def convert_order_dates(orders):
    orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])
    return orders

# Función para fusionar los DataFrames
def merge_dataframes(orders, customers, order_details, products):
    result = (
        orders
        .merge(customers, on='CustomerID', how='left')
        .merge(order_details, on='OrderID', how='inner')
        .merge(products, on='ProductID', how='left')
    )
    return result

# Función para obtener la fecha de la compra anterior
def get_previous_order_date(row, orders_df):
    previous_orders = orders_df[
        (orders_df['CustomerID'] == row['CustomerID']) & (orders_df['OrderDate'] < row['OrderDate'])
    ]
    return previous_orders['OrderDate'].max() if not previous_orders.empty else pd.NaT

# Función para calcular diferencias y monto de venta
def calculate_differences_and_amount(result):
    result['diferencia_dias'] = (result['OrderDate'] - result['OrderDate_anterior']).dt.days
    result['diferencia_meses'] = (result['diferencia_dias'] // 30).fillna(0).astype(int)
    result['monto_venta'] = ((result['UnitPrice_x'] - (result['Discount'] * result['UnitPrice_x'])) * result['Quantity']).fillna(0)
    return result

# Función para agrupar los detalles sin reducir demasiado las filas
def group_order_details(result):
    grouped_result = (
        result
        .groupby(
            ['OrderID', 'ContactName', 'OrderDate'],  # Quitamos 'OrderDate_anterior' del agrupamiento
            as_index=False
        )
        .agg(
            diferencia_dias=('diferencia_dias', 'first'),
            diferencia_meses=('diferencia_meses', 'first'),
            monto_venta=('monto_venta', 'sum')
        )
    )
    return grouped_result

# Función principal
def main(orders, customers, order_details, products):
    orders = convert_order_dates(orders)
    
    result = merge_dataframes(orders, customers, order_details, products)

    # Obtener la fecha de la compra anterior
    result['OrderDate_anterior'] = result.apply(lambda row: get_previous_order_date(row, orders), axis=1)
    
    result = calculate_differences_and_amount(result)

    final_result = group_order_details(result)

    # Renombrar columnas
    final_result.rename(columns={
        'OrderDate': 'fecha_venta',
        'OrderDate_anterior': 'fecha_venta_anterior',
        'ContactName': 'cliente'
    }, inplace=True)

    return final_result

# Ejecutar la función principal
final_result = main(orders, customers, order_details, products)

# Mostrar el resultado final
display(final_result)


,OrderID,cliente,fecha_venta,diferencia_dias,diferencia_meses,monto_venta
0,10248,Paul Henriot,1996-07-04,NaN,0,440.000
1,10249,Karin Josephs,1996-07-05,NaN,0,1863.400
2,10250,Mario Pontes,1996-07-08,NaN,0,1552.600
3,10251,Mary Saveley,1996-07-08,NaN,0,654.060
4,10252,Pascale Cartrain,1996-07-09,NaN,0,3597.900
5,10253,Mario Pontes,1996-07-10,2.0,0,1444.800
6,10254,Yang Wang,1996-07-11,NaN,0,556.620
7,10255,Michael Holz,1996-07-12,NaN,0,2490.500
8,10256,Paula Parente,1996-07-15,NaN,0,517.800
9,10257,Carlos Hernández,1996-07-16,NaN,0,1119.900
